# Fine tuning averaging weights

Fine tuning models could dramatically enhance model's performance.

In the last notebook we have been trying to get the best of the blending technique by changing weights manually.
During this notebook we will write an optimization function.

In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
INPUT_DIR="../data-nlp/input/"
OUTPUT_DIR="../data-nlp/output/"

RAND=10

## Generating a new dataframe from basemodels

In [3]:
files = glob.glob(OUTPUT_DIR+"02_basemodel*.csv")

df = None

for f in files:
    if df is None:
        df = pd.read_csv(f)
    else:
        temp_df = pd.read_csv(f)
        df = df.merge(temp_df, on="id", how="left")

df.head(10)

,id,sentiment_x,kfold_x,lr_cnt_pred,Unnamed: 0,sentiment_y,kfold_y,lr_pred,sentiment,kfold,rf_svd_pred
0,11703_9,1,0,9.996652e-01,0,1,0,0.868421,1,0,0.57
1,6742_8,1,0,9.866726e-01,1,1,0,0.727763,1,0,0.59
2,10071_1,0,0,4.653006e-01,2,0,0,0.441153,0,0,0.54
3,9841_7,1,0,9.999992e-01,3,1,0,0.943808,1,0,0.73
4,11579_10,1,0,9.851556e-01,4,1,0,0.900753,1,0,0.51
5,5216_8,1,0,9.852354e-01,5,1,0,0.729515,1,0,0.51
6,1683_7,1,0,6.576467e-03,6,1,0,0.309059,1,0,0.40
7,818_4,0,0,9.940245e-01,7,0,0,0.713800,0,0,0.73
8,11779_3,0,0,2.116804e-02,8,0,0,0.307074,0,0,0.32
9,27_1,0,0,2.024757e-11,9,0,0,0.001033,0,0,0.12


## Selecting the newly generated features

In [4]:
pred_features = ["lr_pred", "lr_cnt_pred", "rf_svd_pred"]

In [5]:
from sklearn.metrics import roc_auc_score

targets = df.sentiment.values

## Printing  initial AUC values for different basemodels

In [6]:
for col in pred_features:
    auc = roc_auc_score(targets, df[col].values)
    print(f"{col}, overall_auc={auc}")



lr_pred, overall_auc=0.9557241343999999
lr_cnt_pred, overall_auc=0.9462495167999999
rf_svd_pred, overall_auc=0.8796757056


## Importing essential metamodel along with preprocessing tool for scaling

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

## Finding the best coefficients for each fold

In [8]:
def run_training(pred_df, fold):

    train_df = pred_df[pred_df.kfold != fold].reset_index(drop=True)
    valid_df = pred_df[pred_df.kfold == fold].reset_index(drop=True)
    
    xtrain = train_df[pred_features].values
    xvalid = valid_df[pred_features].values

    scaler = StandardScaler()

    xtrain = scaler.fit_transform(xtrain)
    xvalid = scaler.fit_transform(xvalid)

    opt = LogisticRegression()
    opt.fit(xtrain, train_df.sentiment.values)
    preds = opt.predict_proba(xvalid)[:, 1]

    auc = roc_auc_score(valid_df.sentiment.values, preds)
    print(f"fold={fold}, auc={auc}\n")

    valid_df.loc[:, "opt_pred"] = preds

    return opt.coef_

## Appending and averaging coefs

In [9]:
coefs = []
for i in range(5):
    coefs.append(run_training(df, i))
    

fold=0, auc=0.95908304

fold=1, auc=0.95926736

fold=2, auc=0.9545516800000001

fold=3, auc=0.9567231999999999

fold=4, auc=0.95764816



In [10]:
coefs = np.array(coefs)
print(coefs)

[[[ 2.68522025  0.51956405 -0.21366432]]

 [[ 2.66101237  0.52545674 -0.18604012]]

 [[ 2.75142112  0.49555306 -0.20883976]]

 [[ 2.62593371  0.53676715 -0.13553205]]

 [[ 2.66026267  0.54259277 -0.2097813 ]]]


In [11]:
coefs = np.mean(coefs, axis=0)
print(coefs)

[[ 2.67677002  0.52398675 -0.19077151]]


## Final AUC value
The AUC value has been improved a lot comparing to the one in the previous notebook when we chose arbitrary coefs

In [12]:
wt_avg = coefs[0][0] * df.lr_pred.values + coefs[0][1] * df.lr_cnt_pred.values + coefs[0][2] * df.rf_svd_pred.values

print("Optimal AUC after finding coeffecients")
roc_auc_score(targets, wt_avg)

Optimal AUC after finding coeffecients


0.9575652224000001

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b1ab3b60-9130-40c7-8fa2-288950ad463c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>